<a href="https://colab.research.google.com/github/maulanazx23/MuhammadDimasMaulana-2109076022/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import random

class Algoritma_genetika:

    jumlah_chrom = 6  # jumlah chromosome
    gen = ["a", "b", "c", "d"]  # nilai gen abcd
    nilai_per_gen = {
        'min': 0,
        'max': 10
    }  # nilai per gen 0-30
    crossover_rate = 25 / 100.0  # persen
    mutasi_rate = 10 / 100.0  # persen
    total_generasi = 50
    next_gen = np.arange(4)
    stop = False

    def __init__(self):
        # pembentukan chromosome random
        self.first_chromosome = np.random.randint(low=self.nilai_per_gen['min'],
                                                  high=self.nilai_per_gen['max'],
                                                  size=(self.jumlah_chrom, len(self.gen)))
        print(self.first_chromosome)
        print("------------------------------------------------")

    def evaluasi_chrom(self, chrom, generasi):
        # evaluasi chromosome
        # hitung fungsi_objektif dari chromosome yang telah dibuat
        # fungsi_objektif(chromosome) = | (a+2b+3c+4d) – 30 |
        print("GENERASI [" + str(generasi) + "]")

        jumlah_chromo = len(chrom)
        j = np.arange(jumlah_chromo)
        fitness = np.arange(jumlah_chromo, dtype='f')

        for x in range(len(chrom)):
            # FUNGSI OBJEKTIF
            ev = abs((chrom[x][0] + 2 * chrom[x][1] + 3 * chrom[x][2] + 4 * chrom[x][3]) - 30)
            j[x] = ev

            # SELEKSI CHROMOSOME, DAN MENCARI FITNESS
            fitn = 1 / (ev + 1)  # rumus fungsi fitness

            fitness[x] = fitn
            if fitn == 1:
                self.stop = True
            print("CHROMOSOME {0} : {1}, fitness = {2}".format(x, np.array2string(chrom[x], separator=','), fitn))
        print("FITNESS DONE")
        print(j)

        # cari PROBABILITAS (P)
        P = np.arange(jumlah_chromo, dtype='f')
        total_fitness = fitness.sum()
        P = fitness / total_fitness
        print("total fitness:", total_fitness)
        print("Rata-rata fitness:", np.average(fitness))
        print("Probabilitas:", np.array2string(P, separator=','))
        print("Probabilitas PALING TINGGI:", P[P.argmax()], "pada chromosome", P.argmax())
        print("CHROMOSOME YANG MUNGKIN TERPILIH:", np.array2string(chrom[P.argmax()], separator=','), end="\n\n")

        self.next_gen = chrom[P.argmax()]

        # seleksi dengan ROULETTE WHEEL (C) cumulative probabilitas
        C = np.arange(jumlah_chromo, dtype='f')
        total_x = 0
        for x in range(len(P)):
            total_x += P[x]
            C[x] = total_x

        # putar ROULETTE WHEEL sebanyak jumlah sel
        R = np.random.sample(len(fitness))
        new_chrom = np.arange(jumlah_chromo * len(self.gen)).reshape(jumlah_chromo, len(self.gen))

        # CHROMOSOME BARU BERDASARKAN ROULETTE WHEEL
        for y in range(len(R)):
            for k in range(len(new_chrom)):
                if R[y] < C[0]:
                    new_chrom[y] = chrom[0]
                elif (C[k - 1] < R[y]) and (R[y] < C[k]):
                    new_chrom[y] = chrom[k]

        # CROSSOVER, mencari crossover
        R = np.random.sample(jumlah_chromo)
        index_chrom_parent = []  # [1,2,3, ...]
        for p in range(len(R)):
            if R[p] < self.crossover_rate:
                index_chrom_parent.append(p)

        # MENENTUKAN POSISI CROSS OVER
        # membangkitkan bilangan acak dari 1 sampai (panjang chromosome - 1)
        posisi_CO = np.random.randint(low=1, high=len(self.gen), size=len(index_chrom_parent))

        # PROSES CROSSOVER, pertukaran cut-point
        off_spring = np.arange(len(self.gen) * len(index_chrom_parent)).reshape(len(index_chrom_parent), len(self.gen))

        for i_parent in range(len(index_chrom_parent)):
            index_chrome_1 = index_chrom_parent[i_parent]
            if i_parent == len(index_chrom_parent) - 1:
                index_chrome_2 = index_chrom_parent[0]
            else:
                index_chrome_2 = index_chrom_parent[i_parent + 1]
            # melakukan cut-point
            cut_point = posisi_CO[i_parent]
            for p in range(len(new_chrom[index_chrome_1])):
                # LOOPING BERDASARKAN GEN
                if p >= posisi_CO[i_parent]:
                    # JIKA GEN[P] LEBIH BESAR ATAU SAMA DENGAN BILANGAN ACAK[P],
                    # MAKA DIGANTI DENGAN CHROMOSOME KE-2
                    off_spring[i_parent][p] = new_chrom[index_chrome_2][p]
                else:
                    off_spring[i_parent][p] = new_chrom[index_chrome_1][p]

        # PROSES CROSSOVER DISIMPAN PADA VARIABEL 'off_spring'
        # MELAKUKAN PENGGABUNGAN OFFSPRING DENGAN NEW_CHROM
        for x in range(len(off_spring)):
            new_chrom[index_chrom_parent[x]] = off_spring[x]

        # PROSES MUTASI
        total_gen = len(chrom) * len(chrom[0])
        jumlah_mutasi = self.mutasi_rate * total_gen
        jumlah_mutasi = int(jumlah_mutasi)

        random_i_mutasi = np.random.randint(low=0, high=total_gen, size=jumlah_mutasi)

        for x in range(len(random_i_mutasi)):
            index_mutasi = random_i_mutasi[x]
            banyak_kromosom = len(chrom)
            banyak_gen = len(chrom[0])
            random_value = random.randint(self.nilai_per_gen['min'], self.nilai_per_gen['max'])
            if index_mutasi <= banyak_gen:
                # jika index_mutasi <= banyak gen, maka akan mengganti
                # gen pada chromosome baru yang ke-0
                new_chrom[0][index_mutasi - 1] = random_value
            else:
                # POSISI Y DARI KROMOSOM, UNTUK MENCARI INDEX
                pos_y = index_mutasi / banyak_gen
                pos_y = int(pos_y)
                pos_x = index_mutasi % banyak_gen
                new_chrom[pos_y][pos_x] = random_value
        return new_chrom

    def do_now(self):
        chromosome_current = self.first_chromosome
        for generasi in range(0, self.total_generasi):
            if not self.stop:
                chromosome_current = self.evaluasi_chrom(chromosome_current, generasi)
        print("DONE")
        print("CHROMOSOME TERTINGGI ADALAH")
        print(self.next_gen)

run = Algoritma_genetika()
run.do_now()


[[1 3 0 6]
 [6 3 3 0]
 [4 6 6 6]
 [8 0 4 3]
 [6 4 4 1]
 [0 2 4 4]]
------------------------------------------------
GENERASI [0]
CHROMOSOME 0 : [1,3,0,6], fitness = 0.5
CHROMOSOME 1 : [6,3,3,0], fitness = 0.1
CHROMOSOME 2 : [4,6,6,6], fitness = 0.034482758620689655
CHROMOSOME 3 : [8,0,4,3], fitness = 0.3333333333333333
CHROMOSOME 4 : [6,4,4,1], fitness = 1.0
CHROMOSOME 5 : [0,2,4,4], fitness = 0.3333333333333333
FITNESS DONE
[ 1  9 28  2  0  2]
total fitness: 2.3011494
Rata-rata fitness: 0.3835249
Probabilitas: [0.21728273,0.04345655,0.01498502,0.14485516,0.43456545,0.14485516]
Probabilitas PALING TINGGI: 0.43456545 pada chromosome 4
CHROMOSOME YANG MUNGKIN TERPILIH: [6,4,4,1]

DONE
CHROMOSOME TERTINGGI ADALAH
[6 4 4 1]
